In [10]:
import pandas as pd
def ptr2point(excel,area):

  if area == 'paringin':
    mitra = 'BUMA'
    sheet = 'COAL BUMA'
    skip = 7
    column_no = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
    nama_field = ['date','class_unit','bdy_plan_weekly','bdy_plan_monthly','unit','seam_detail','seam','series','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','rom','access','highest','lowest','v_dist','st_dist','hz_dist','truck_count']
  if area  == 'south':
    sheet='COAL BUMA'
    mitra = 'BUMA'
    skip=8
    column_no = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
    nama_field = ['date','class_unit','bdy_plan_weekly','bdy_plan_monthly','unit','seam_detail','seam','series','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','rom','access','highest','lowest','v_dist','st_dist','hz_dist','truck_count']
  if area  == 'wara':
    sheet='COAL WARA'
    mitra = 'SIS'
    skip=5
    column_no = [0,1,2,3,4,5,6,7,8,9,10,11,12,15,17,18]
    nama_field = ['date','seam','series','bdy_plan','unit','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','rom','access','v_dist','hz_dist','truck_count']
  if area  == 'central':
    sheet='COAL CT'
    mitra = 'SIS'
    skip=5
    column_no = [0,1,2,4,5,6,7,8,9,10,11,12,13,16,18,51]
    nama_field = ['date','seam','series','unit','lp_n','lp_e','lp_z','bdy_plan','dp_n','dp_e','dp_z','rom','access','v_dist','hz_dist','truck_count']

  df_ptr = pd.read_excel(excel, sheet, skiprows=skip, usecols=column_no, header=0)
  df_ptr.columns = nama_field
  df_ptr = df_ptr.dropna(subset=['date'])
  df_ptr['week'] = (df_ptr['date']- pd.DateOffset(days=1)).dt.isocalendar().week #add week column
  df_ptr['unit'] = df_ptr.unit.str.strip()
  df_ptr['unit'] = df_ptr.unit.str[:2] + '-' + df_ptr.unit.str[-2:] #remove -00 from unit
  df_ptr.fillna(0,inplace=True)
  df_ptr['vol_hd'] = df_ptr['hz_dist'] * df_ptr['truck_count']
  df_ptr['vol_vd'] = df_ptr['v_dist'] * df_ptr['truck_count']
  df_ptr['hz_dist_mtd'] = df_ptr['vol_hd'].cumsum()/df_ptr['truck_count'].cumsum()
  df_ptr['v_dist_mtd'] = df_ptr['vol_vd'].cumsum()/df_ptr['truck_count'].cumsum()
  df_ptr['pit'] = area
  df_ptr['mitra kerja'] = mitra
  last_update = df_ptr['date'].max().strftime('%y%m%d')
  df_ptr[['date','unit','seam','series','lp_n','lp_e','lp_z','rom','access','v_dist','hz_dist','truck_count','pit','mitra kerja']].to_csv('coal_ptr_'+area+'_'+last_update+'.csv', index=True, index_label='id')
  csvt_txt = "\"Integer\",\"Date\",\"String\",\"String\",\"String\",\"CoordY\",\"CoordX\",\"Real\",\"String\",\"String\",\"Real\",\"Real\",\"Real\",\"String\",\"String\""
  csvt_file = open('coal_ptr_'+area+'_'+last_update+'.csvt', "w")
  n = csvt_file.write(csvt_txt)
  csvt_file.close()

  print('Selesai, silakan cek file untuk download')

In [11]:
!pip install pyproj
import math
from pyproj import Proj

UTM50s = Proj('+proj=utm +zone=50 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
#parameter site calibration Tutupan
rot,dx,dy,sf,ox,oy,olocx,olocy = 57.26, -330718.744,-9748020.012, 1.00008025, 334759.446 ,9757090.68,4040.702,9070.668
rot_p,dx_p,dy_p,sf_p,ox_p,oy_p,olocx_p,olocy_p = 23.05789276,	-331701.558,	-9741692.346,	1.000039197,	332681.896,	9746705.39,	980.338,	5013.044


def Ttp2Lon(easting,northing):
    x1 = (easting-olocx)/sf+olocx
    y1 = (northing-olocy)/sf+olocy
    x2 = x1-dx
    y2 = y1-dy
    x3 = (x2-ox)*math.cos(math.radians(-rot)) - (y2-oy)*math.sin(math.radians(-rot)) + ox
    y3 = (y2-oy)*math.cos(math.radians(-rot)) + (x2-ox)*math.sin(math.radians(-rot)) + oy
    geo = UTM50s(x3,y3,inverse=True)
    return geo[0]
def Ttp2Lat(easting,northing):
    x1 = (easting-olocx)/sf+olocx
    y1 = (northing-olocy)/sf+olocy
    x2 = x1-dx
    y2 = y1-dy
    x3 = (x2-ox)*math.cos(math.radians(-rot)) - (y2-oy)*math.sin(math.radians(-rot)) + ox
    y3 = (y2-oy)*math.cos(math.radians(-rot)) + (x2-ox)*math.sin(math.radians(-rot)) + oy
    geo = UTM50s(x3,y3,inverse=True)
    return geo[1]

def Ttp2Geo(easting,northing):
    x1 = (easting-olocx)/sf+olocx
    y1 = (northing-olocy)/sf+olocy
    x2 = x1-dx
    y2 = y1-dy
    x3 = (x2-ox)*math.cos(math.radians(-rot)) - (y2-oy)*math.sin(math.radians(-rot)) + ox
    y3 = (y2-oy)*math.cos(math.radians(-rot)) + (x2-ox)*math.sin(math.radians(-rot)) + oy
    geo = UTM50s(x3,y3,inverse=True)
    return geo

def Prg2Geo(easting,northing):
    x1 = (easting-olocx_p)/sf_p+olocx_p
    y1 = (northing-olocy_p)/sf_p+olocy_p
    x2 = x1-dx_p
    y2 = y1-dy_p
    x3 = (x2-ox_p)*math.cos(math.radians(-rot_p)) - (y2-oy_p)*math.sin(math.radians(-rot_p)) + ox_p
    y3 = (y2-oy_p)*math.cos(math.radians(-rot_p)) + (x2-ox_p)*math.sin(math.radians(-rot_p)) + oy_p
    geo = UTM50s(x3,y3,inverse=True)
    return geo

In [12]:
Prg2Geo(5200,100)

(115.51288114773563, -2.346676346571078)